First, move within a simulation directory. This directory should contains the sub-directory 'output/' that contains snapshot files, and a file 'snapshot_times.txt' that lists all snapshot scale-factors/redshifts/times in the simulation and their corresponding file index number.

Ensure that gizmo_analysis and utilities directories are in your python path, then...

In [ ]:
import gizmo_analysis as gizmo
import utilities as ut

In [ ]:
# you can access the files as named or use the aliases in __init__.py to keep it shorter 
# for example, these are the same:

gizmo.gizmo_io
gizmo.io

# read particle data

In [ ]:
# read star and dark-matter particles at z = 0

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0)

In [ ]:
# alternately, read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0)

In [ ]:
# this tutorial assumes that you are in the root directory of a simulation, 
# but you can read a simulation at any location using input argument 'simulation_directory':

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0, simulation_directory='/arb/it/rary/loc/a/tion')

In [ ]:
# each particle species is stored as its own dictionary
# 'star' = stars, 'gas' = gas, 'dark' = dark matter, 'dark.2', 'dark.3', etc = low-resolution dark matter

part.keys()

In [ ]:
# properties of star particles are stored as dictionary

part['star'].keys()

In [ ]:
# properties of dark matter particles are stored as dictionary

part['dark'].keys()

# particle properties

In [ ]:
# 3-D position of star particles (particle.number x dimension.number array) [Mpc comoving]

part['star']['position']

In [ ]:
# 3-D velocity of star particles (particle_number x dimension_number array) [km/s physical]

part['star']['velocity']

In [ ]:
# mass of star particles [M_sun]

part['star']['mass']

In [ ]:
# formation scale-factor of star particles

part['star']['form.scalefactor']

In [ ]:
# use .prop() to compute derived quantities,
# such as the age of the Universe [Gyr] when a star particle formed
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

part['star'].prop('form.time')

In [ ]:
# similarly, get the age of a star particle (the lookback time to when it formed) [Gyr]

part['star'].prop('age')

# metallicities

In [ ]:
# 'metallicities' are stored in the catalog as *mass fractions*
# one value for each element, in a particle_number x element_number array
# the first value is the mass fraction of all metals (everything not H, He)
# then He, C, N, O, etc

part['star']['massfraction']

In [ ]:
# get individual elements by their index

# total metal mass fraction (everything not H, He) is index 0
print(part['star']['massfraction'][:, 0])

# iron is index 10
print(part['star']['massfraction'][:, 10])

In [ ]:
# alternately use .prop() to compute derived quantities,
# including calling elements by their name or symbol
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

print(part['star'].prop('massfraction.metals'))
print(part['star'].prop('massfraction.carbon'))
print(part['star'].prop('massfraction.iron'))
print(part['star'].prop('massfraction.fe'))  # can use name or symbol

In [ ]:
# also use .prop() to compute metallicity [Z/H]
# for example, iron abundance [Fe/H] :=
#   log10((mass_iron / mass_hydrogen)_particle / (mass_iron / mass_hydrogen)_sun)
# my pipeline assumes solar abundances from Asplund et al 2009

print(part['star'].prop('metallicity.total'))
print(part['star'].prop('metallicity.fe'))

In [ ]:
# also use .prop() to compute simple arithmetic combinations, such as [Mg/Fe]

part['star'].prop('metallicity.mg - metallicity.fe')

In [ ]:
# refer to ut.basic.constants for assumed solar values (Asplund et al 2009) and other constants

ut.basic.constants.sun_composition

# additional information stored in sub-dictionaries

In [ ]:
# dictionary of useful information about the simulation

part.info

In [ ]:
# dictionary of information about this snapshot's index, scale-factor, redshift, time, lookback-time

part.snapshot

In [ ]:
# dictionary class of arrays about *all* snapshots that were saved for the simulation

print(part.Snapshot.keys())
print(part.Snapshot['redshift'][:10])

In [ ]:
# dictionary class of cosmological parameters, with function for cosmological conversions

part.Cosmology

In [ ]:
# position [kpc comoving] and velocity [km/s physical] of the center of the host galaxy
# this was computed during read in, using ut.particle.get_center_position() and ut.particle.get_center_velocity()
# functions in gizmo.analysis use these values in computing profiles

print(part.center_position)
print(part.center_velocity)

See gizmo.analysis for examples of high-level analysis, including plotting these data.

See ut.particle for mid-level analysis functions that may be useful.

See other modules within utilities for low-level functions that may be useful.

# principal (major and minor axes) of stellar disk

In [ ]:
# can compute and assign principal axes (moment of inertia tensor) of stars during read in as below

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, assign_principal_axes=True)

In [ ]:
# moment of inertia tensor is stored similar to center_position

print(part.principal_axes_vectors)

In [ ]:
# now can compute different types of distances of star particles from center of main galaxy
# compute 3-D distance from primary galaxy center along simulation's default x,y,z axes [kpc physical]

part['star'].prop('host.distance')

In [ ]:
# compute total (scalar) distance

part['star'].prop('host.distance.total')

In [ ]:
# compute 3-D distance aligned with the principal (major, intermediate, minor) axes

part['star'].prop('host.distance.principal')

In [ ]:
# compute 3-D distances aligned with the principal axes in cylindrical coordinates
# first value is along the major axes (R, positive definite)
# second value is vertical height wrt the disk (Z, signed)
# third value is angle (phi, 0 to 2 * pi)

part['star'].prop('host.distance.principal.cylindrical')

In [ ]:
# same for velocity
# compute 3-D velocity from main galaxy center along simulation's default x,y,z axes [km / s]

part['star'].prop('host.velocity')

In [ ]:
# compute 1-D velocity [km / s]

part['star'].prop('host.velocity.total')

In [ ]:
# compute 3-D velocity along the principal (major, intermediate, minor) axes [km / s]

part['star'].prop('host.velocity.principal')

In [ ]:
# compute 2-D velocity along the major axes (major + intermediate) and minor axis
# first value is along the major axes (positive definite)
# second value is vertical velocity wrt the disk (signed)

part['star'].prop('host.velocity.principal.cylindrical')

# particle tracking

Some simulations have pre-compiled HDF5 files to help with particle tracking. These are stored in the directory 'track/' (if present). The code that generates and reads these files is in gizmo_track.py.

star\_indices\_*.hdf5 files store, for each star particles at z = 0, a pointer to where it was in the catalog at each previous snapshot (replace * with snapshot index). This makes it easy to quickly get the properties of a given star particle at any previous snapshot. These pointers are stored in an HDF5 file, one for each previous snapshot.

In [1]:
# first, read catalog of star particles at z = 0

part_at_z0 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 0)


# in utilities.simulation.Snapshot():
  read snapshot_times.txt
  input redshift = 0.000 -> snapshot index = 600
  reading snapshot index = 600, redshift = 0.000

# in gizmo_analysis.gizmo_io.Read():
* read header from: output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
  dark      (id = 1): 70514272 particles
  dark.2    (id = 2): 5513331 particles
  gas       (id = 0): 57060074 particles
  star      (id = 4): 13976485 particles
  blackhole (id = 5): 0 particles

* read particles
  from: snapshot_600.0.hdf5
  from: snapshot_600.1.hdf5
  from: snapshot_600.2.hdf5
  from: snapshot_600.3.hdf5

* read cosmological parameters from: initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (41792.125, 44131.215, 46267.672) [kpc comoving]
  velocity = (-52.5, 71.9, 95.2) [km / s]



In [ ]:
# say that you want to find out what they were doing at z = 1
# then, read in catalog of star particles at z = 1 (snapshot 277)

part_at_z1 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 1)

In [ ]:
# use the function within gizmo_track.py to read star index pointers associated with the catalog z = 1

gizmo.track.IndexPointer.io_index_pointer(part_at_z1)

In [ ]:
# pointers are stored via numpy array appended to particle dictionary at the relevant snapshot
# a negative value means that the star formed after this snapshot (so it does not exist at this snapshot)

part_at_z1.index_pointers

In [3]:
# so, say that you have a list of the indices of star particles of interest at z = 0

indices_at_z0 = np.array([3, 5, 8, 13])

In [4]:
# their positions at z = 0

part_at_z0['star']['position'][indices_at_z0]

array([[ 43404.91796875,  44238.88671875,  46855.734375  ],
       [ 43406.32421875,  44237.12109375,  46849.92578125],
       [ 41799.5546875 ,  44130.66796875,  46263.68359375],
       [ 41799.5546875 ,  44130.66015625,  46263.56640625]], dtype=float32)

In [ ]:
# get their indices in the catalog at z = 1

indices_at_z1 = part_at_z1.index_pointers[indices_at_z0]
print(indices_at_z1)

In [ ]:
# now you easily can get any property of interest at z = 1, for example, positions

part_at_z1['star']['position'][indices_at_z1]

Another useful file: star\_form\_host\_distance\_600.hdf5 stores, for each star particle at z = 0, its 3-D distance at the first snapshot after it formed (formation distance) wrt to the main host galaxy, in [kpc physical]. These distances are aligned with the major, intermediate, minor axes of the stellar disk (as defined via its moment of inertia tensor) at that snapshot.

In [ ]:
# use the function within gizmo_track.py to read this file and assign values directly to the catalog at z = 0

gizmo.track.HostDistance.io_formation_coordinates(part_at_z0)

In [5]:
# alternately, read formation distances at the same time read snapshot itself
# (as with principal axes)

part_at_z0 = gizmo.io.Read.read_snapshots(
    ['star'], 'redshift', 0, assign_principal_axes=True, assign_formation_coordinates=True)


# in utilities.simulation.Snapshot():
  read snapshot_times.txt
  input redshift = 0.000 -> snapshot index = 600
  reading snapshot index = 600, redshift = 0.000

# in gizmo_analysis.gizmo_io.Read():
* read header from: output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
  dark      (id = 1): 70514272 particles
  dark.2    (id = 2): 5513331 particles
  gas       (id = 0): 57060074 particles
  star      (id = 4): 13976485 particles
  blackhole (id = 5): 0 particles

* read particles
  from: snapshot_600.0.hdf5
  from: snapshot_600.1.hdf5
  from: snapshot_600.2.hdf5
  from: snapshot_600.3.hdf5

* read cosmological parameters from: initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (41792.125, 44131.215, 46267.672) [kpc comoving]
  velocity = (-52.5, 71.9, 95.2) [km / s]

* assigning principal axes of galaxy/halo:
  axis ratios: min/maj = 0.503, min/med = 0.509, med/maj 

In [6]:
# 3-D distance at formation, aligned with the principal axes of the host galaxy at that time [kpc physical]
# principal axes defined according to all star particles within the host galaxy, independently at each snapshot
# distance along dimension 0 is aligned with the major axis
# distance along dimension 1 is algined with the intermediate axis
# distance along dimension 2 is aligned with the minor (Z) axis

part_at_z0['star']['form.host.distance']

array([[ -1.22573435e+00,  -3.16812187e-01,   1.52569735e+00],
       [  1.93979454e+01,   7.89692259e+00,  -3.43014755e+01],
       [ -5.79297607e+02,  -8.96287460e+01,   1.60383392e+02],
       ..., 
       [ -9.05316467e+01,  -3.17012157e+01,  -2.47149734e+01],
       [ -1.34268036e+01,  -5.74990749e+00,  -1.78105259e+00],
       [ -1.34432344e+01,  -5.75029564e+00,  -1.73416471e+00]], dtype=float32)

In [7]:
# total scalar (absolute) distance wrt the host galaxy at formation [kpc physical]
# this is a derived quantity, so need to call via .prop()

part_at_z0['star'].prop('form.host.distance.total')

array([   1.98256075,   40.18996048,  607.73504639, ...,   99.05441284,
         14.71436787,   14.72391891], dtype=float32)

In [8]:
# 3-D array of formation distance wrt the host galaxy in cylindrical coordinates [kpc physical]

part_at_z0['star'].prop('form.host.distance.cylindrical')

array([[  1.26601529e+00,   1.52569735e+00,   3.39452457e+00],
       [  2.09437733e+01,  -3.43014755e+01,   3.86612892e-01],
       [  5.86190247e+02,   1.60383392e+02,   3.29509544e+00],
       ..., 
       [  9.59215622e+01,  -2.47149734e+01,   3.47841668e+00],
       [  1.46061792e+01,  -1.78105259e+00,   3.54620552e+00],
       [  1.46214380e+01,  -1.73416471e+00,   3.54578733e+00]], dtype=float32)

In [13]:
# these value look more reasonable is restrict to star particles that formed within the host galaxy

# select particles formed at d = 0 - 8 kpc physical
part_indices = ut.array.get_indices(part_at_z0['star'].prop('form.host.distance.total'), [0, 8])

part_at_z0['star'].prop('form.host.distance.cylindrical', part_indices)

array([[ 1.26601529,  1.52569735,  3.39452457],
       [ 5.2021389 , -0.05428728,  3.83885074],
       [ 5.19000244,  0.58169037,  2.74372029],
       ..., 
       [ 3.82861757, -5.66393709,  0.53575462],
       [ 7.15072107, -0.22851086,  4.99534464],
       [ 7.38440609,  0.08358712,  4.96902847]], dtype=float32)

# profile of property

A common task is to compute a radial profile of a given quantity, such as mass density, average age, median metallicity, etc.

The high-level functions below make this easier to do.

In [ ]:
# first, initiate an instance of SpeciesProfileClass
# as you initialize,choose your distance/radius binning scheme: 
#   'log' v 'linear', distance limits, bin width, number of spatial dimensions of profile
# refer to ut.binning.DistanceBinClass() for more

# linear binning from 0 to 20 kpc with 1 kpc bin width, assuming a 3-D profile
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='linear', limits=[0, 20], width=1, dimension_number=3)

In [ ]:
# using this binning scheme,
# compute sum/histogram/density of mass of star particles in each bin
# this returns a bunch of summed properties via a dictionary

pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass')

In [ ]:
# in principle, you can supply a list of multiple species, and it will compute profiles for each
# thus, it returns a dictionary for each species

pro.keys()

In [ ]:
# the quantities that it stores in each bin

pro['star'].keys()

In [ ]:
# alternately, you may want to compute profiles along a disks R or Z axes
# if so, first define the dimensionality of the profile when you initiate the class

# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 2-D profile (along R)
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='log', limits=[0.1, 10], width=0.1, dimension_number=2)

In [ ]:
# set rotation = True to force it to compute profiles along the principal axes (assuming that you read them in)
# use other_axis_distance_limits to limit the extent along the other axis, 
#   in this case, limit the Z axis to within +/- 1 kpc (in the profile, all distances are absolute)

pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass', rotation=True, other_axis_distance_limits=[0, 1])

In [ ]:
# similarly, do this to compute profiles along Z

# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 1-D profile (along Z)
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='log', limits=[0.1, 10], width=0.1, dimension_number=1)

# limit the R axex to [5, 8] kpc 
pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass', rotation=True, other_axis_distance_limits=[5, 8])

In [ ]:
# using the same binning scheme
# this function computes various statistics of a property of star particles in each bin
# by default, it weights the property by the mass of each particle
# this returns a bunch of statistics via a dictionary

pro = SpeciesProfile.get_statistics_profiles(
    part, 'star', 'age', weight_by_mass=True, rotation=True, other_axis_distance_limits=[5, 8])

In [ ]:
# the quantities that it stores in each bin

pro['star'].keys()